In [ ]:
stlv = 521
version = 211

max_anzahl_im_aufmass = 30


In [ ]:
'''
SQL
select count(*) anzahl, p.pschluessel, p.bl, p.la, p.text from leistung l
join position2 p on p.referenz_id = l.leistung_id
where l.stlv=521 and l.version='211'
group by p.pschluessel, p.text, p.bl, p.la
order by anzahl desc;
'''

# Connect to Database
import os

database_url = os.environ["database_url"]
verbose = False

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

session = sessionmaker(bind=create_engine(database_url, echo=False), autoflush=False)()

In [ ]:
from kolonne.database.models import Position2, Leistung
from sqlalchemy import func, desc

leistungen_im_aufmass = session.query(Position2.pschluessel, Position2.bl, Position2.la, Position2.text, func.count(Position2.pschluessel)).select_from(Position2).join(Leistung, Leistung.leistung_id==Position2.referenz_id)\
.filter(Position2.typ == 'L').filter(Leistung.stlv == stlv, Leistung.version == str(version)).group_by(Position2.pschluessel, Position2.bl, Position2.la, Position2.text)\
.order_by(desc(func.count(Position2.pschluessel))).all()

# print(leistungen_im_aufmass)

In [ ]:
data = list()

for l in leistungen_im_aufmass:
    data.append({"Pschlüssel": l[0], "Bl": l[1], "La": l[2], "Text": l[3][0:30], "Anzahl": l[4]})
    if len(data) > max_anzahl_im_aufmass:
        break;

# [{'Pschlüssel': '20.20.022.02.16', 'Anzahl': 482}, ... ]


In [ ]:
# DataFrame erstellen

import pandas as pd

data_frame = pd.DataFrame(data)

print(data_frame)

In [ ]:
import numpy as np

def show_values(axs, orient="v", space=.01):
    def _single(ax):
        if orient == "v":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() / 2
                _y = p.get_y() + p.get_height() + (p.get_height()*0.01)
                value = '{:.1f}'.format(p.get_height())
                ax.text(_x, _y, value, ha="center") 
        elif orient == "h":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() + float(space)
                _y = p.get_y() + p.get_height() - (p.get_height()*0.5)
                value = '{:.1f}'.format(p.get_width())
                ax.text(_x, _y, value, ha="left")

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _single(ax)
    else:
        _single(axs)
        

In [ ]:
# Diagramm erstellen

import matplotlib.pyplot as plt
import seaborn as sn

sn.set(font_scale=1.0)
fig, ax = plt.subplots(figsize=(20,7))

palette = ['tab:green','tab:red']

labels = data_frame.loc[:,'Pschlüssel'].values
ax.set_xticks(range(len(labels)))
ax.set_xticklabels(labels, rotation=60)

p = sn.barplot(data = data_frame
            ,x = 'Pschlüssel'
            ,y = 'Anzahl'
            ,palette=palette
            )

ax.set_title(f"Top {max_anzahl_im_aufmass} Leistungen: {stlv}/{version}", y=1, fontsize = 16)
show_values(p)
